
## Initial testing on a naive bayes classifier yielded about 56% accuracy. 
 we used a reduced set to train the model  due to time constraints. 
 
 further explorations will attempt to improve on this baseline  

In [10]:
import joblib
import pandas as pd
import numpy as np
import logging
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

import re
from bs4 import BeautifulSoup
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/dliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##get the data into pandas for analysis\"\"\"\n",
df = pd.read_csv('rspct.tsv', sep='\t')



reduced = df.sample(frac=.3)


In [11]:
reduced = pd.read_csv('reduced30k.tsv','\t')

reduced.to_csv('reduced30k.tsv', sep='\t')

In [12]:
labels = reduced['subreddit'].drop_duplicates()

In [13]:
labels[:50].values


array(['HomeDepot', 'DunderMifflin', 'hometheater', 'EnterTheGungeon',
       'bigboobproblems', 'cinematography', 'Tinder', 'LearnJapanese',
       'futarp', 'OnePieceTC', 'Firefighting', 'fleshlight', 'lotr',
       'knifeclub', 'sociopath', 'bleach', 'SCCM', 'GhostRecon',
       'Ayahuasca', 'codes', 'preppers', 'grammar', 'NewSkaters',
       'Truckers', 'southpark', 'Dreams', 'JUSTNOMIL', 'bigdickproblems',
       'EternalCardGame', 'evangelion', 'mercedes_benz', 'Cuckold',
       'writing', 'afinil', 'synology', 'thinkpad', 'MDMA', 'sailing',
       'cfs', 'siacoin', 'ASUS', 'OccupationalTherapy', 'biology',
       'thelastofus', 'lonely', 'swrpg', 'acting', 'transformers',
       'vergecurrency', 'Beekeeping'], dtype=object)

In [14]:
df = reduced.sample(frac=.1)
df.shape

(30390, 5)

classes have been balanced! time to clean some of the punctation, and pseudo html from the text


In [15]:

from sklearn.preprocessing import FunctionTransformer

def clean_text(X):
    """
        text: a string
        
        return: modified initial string
    """
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STOPWORDS = set(stopwords.words('english'))
    
    X = X.map(lambda t : BeautifulSoup(t, "lxml").text) # strip html tags
    X = X.map(lambda t : t.lower()) # lowercase text
    X = X.map(lambda t : REPLACE_BY_SPACE_RE.sub(' ', t))  # symbols by space in text
    X = X.map(lambda t : BAD_SYMBOLS_RE.sub('', t)) # delete symbols which are in BAD_SYMBOLS_RE
    X = X.map(lambda t : ' '.join(word for word in t.split() if word not in STOPWORDS))# delete stopwords
    return X


text_transformer = FunctionTransformer(clean_text)


At this point we can consider if we want to prepend the title to the post, or handel it as a separate feature. i think for simplicity, the first time, we will just use the post only then compare to results with title + selftext 

In [16]:
X1 = df['title'].str.cat(df['selftext'], sep=' ')
X = df.selftext
y = df.subreddit


In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report


In [20]:

trans_pipe = Pipeline([
            ('clean', text_transformer),
              ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer())  ## pay attn to transformer vs vectorizer
            ])
X_trans = trans_pipe.fit_transform(X1)        ## should be able to save this for later
X_train,  X_test, y_train,y_test = train_test_split(X_trans,y,test_size=.2,random_state=42)

In [22]:
X_train.shape,  y_train.shape, X_test.shape, y_test.shape

((24312, 157656), (24312,), (6078, 157656), (6078,))

In [23]:
# setup pipeline and fit

sgd =  SGDClassifier(loss='hinge',
                penalty='l2',
                alpha=1e-3,
                random_state=42, 
                max_iter=5, 
                tol=None)
sgd.fit(X_train, y_train)


SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=None, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [24]:

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
class_rep = classification_report(y_test, y_pred)


accuracy 0.6140177690029615


/home/dliu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1206: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dliu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1206: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# randomize search CV for SVc

# C
C = [.01, .1]

# gamma
gamma = [ .001, .01, .1, 1, 10]

# degree
degree = [ 3, 4, 5]

# kernel
kernel = ['linear',  'poly']

probability = [True]


# Create the random grid
random_grid = {'C': C,
              'kernel': kernel,
              'gamma': gamma,
              'degree': degree,
               'probability': probability
             }


In [ ]:
pipe = SVC(random_state=8)
# Definition of the random search
random_search = RandomizedSearchCV(pipe,
                                   param_distributions=random_grid,
                                   n_iter=10,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=5, 
                                   random_state=8,
                                   n_jobs=-1)

# Fit the random search model
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 58.4min


In [ ]:

print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

In [ ]:


# Create the parameter grid based on the results of random search 
C = [.0001, .001, .01, .1]
degree = [3, 4, 5]
gamma = [1, 10, 100]
probability = [True]

param_grid = [
  {'C': C, 'kernel':['linear'], 'probability':probability},
  {'C': C, 'kernel':['poly'], 'degree':degree, 'probability':probability},
  {'C': C, 'kernel':['rbf'], 'gamma':gamma, 'probability':probability}
]

# Create a base model
svc = svm.SVC(random_state=8)

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=svc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search.fit(features_train, labels_train)

### Once we have trained the best model we can get conditional probabilities as well as the prediction: 

The conditional class probabilities can be obtained by typing:

    svc_pred = best_svc.predict_proba(features_test)

In [ ]:

# Definition of the random search
random_search = RandomizedSearchCV(SVC(random_state=8),
                                   param_distributions=random_grid,
                                   n_iter=10,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=4, 
                                   random_state=8)

# Fit the random search model
random_search.fit(X_trans, y_train)

In [ ]:

print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

In [ ]:
#model = sv

y_pred = model.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
class_rep = classification_report(y_test, y_pred,target_names=my_labels)


In [ ]:
my_labels = df['subreddit'].drop_duplicates()  

In [ ]:

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
class_rep = classification_report(y_test, y_pred,target_names=my_labels)

In [ ]:
print(class_rep)

In [ ]:
import joblib
with open('sgdb_joblib.pkl', 'wb') as f:
    joblib.dump(pipe, f, compress=5)

We will use the following to load this model elsewhere:

    with open('nb_self', 'rb') as handle:
        model = pickle.load(handle)
    
    model
